In [111]:
import pandas as pd

In [112]:
import music21 as m21

In [291]:
b = m21.corpus.parse('bwv66.6')
score_info = []
for i, part in enumerate(b.parts):
    for obj in part.flat.getElementsByClass(m21.note.GeneralNote):
        if isinstance(obj, m21.note.Note):
            d = {
                    'part' : i,
                    'id' : obj.id,
                    'pitch_num' : 0,
                    'pitch_class' : obj.pitch.pitchClass,
                    'score_pos' : obj.offset,
                    'duration' : obj.duration.quarterLength
                }
            score_info.append(d)
        elif isinstance(obj, m21.note.Chord):
            for j in obj.pitchClasses:
                d = {
                    'part' : i,
                    'id' : obj.id,
                    'pitch_num' : j,
                    'pitch_class' : obj.pitch.pitchClass,
                    'score_pos' : obj.offset,
                    'duration' : obj.duration.quarterLength
                }
                score_info.append(d)

df = pd.DataFrame(score_info)
df[df['part'] == 2].head()
a = 0.5
b = 2.0

In [327]:
edges = {
    (0,0) : ((0,0),(1,1)),
    (0,1) : ((0,0),),
    (0,2) : ((0,1),),
    (0,3) : ((1,0),),
    (0,4) : ((0,0),),
    (0,5) : ((0,0),),
    (0,6) : None,
    (0,7) : ((0,1),),
    (0,8) : ((0,0),),
    (0,9) : ((0,1),),
    (0,10): ((0,0),(1,1)),
    (0,11): ((0,1),(1,0)),
    
    (1,1) : ((0,0),(1,1)),
    (1,2) : ((0,0),),
    (1,3) : ((0,0),),
    (1,4) : ((0,0),),
    (1,5) : ((0,0),),
    (1,6) : ((0,0),),
    (1,7) : None,
    (1,8) : ((0,0),),
    (1,9) : ((0,0),),
    (1,10): ((0,0),),
    (1,11): ((0,0),),
    
    (2,2) : ((0,0),(1,1)),
    (2,3) : ((1,0),),
    (2,4) : ((0,0),),
    (2,5) : ((0,0),),
    (2,6) : ((0,0),),
    (2,7) : ((0,0),),
    (2,8) : None,
    (2,9) : ((0,0),),
    (2,10): ((1,0),),
    (2,11): ((0,0),),
    
    (3,3) : ((0,0),(1,1)),
    (3,4) : ((0,0),),
    (3,5) : ((0,0),),
    (3,6) : ((0,0),),
    (3,7) : ((0,1),),
    (3,8) : ((0,0),),
    (3,9) : None,
    (3,10): ((0,0),),
    (3,11): ((0,0),),
    
    (4,4) : ((0,0),(1,1)),
    (4,5) : ((0,0),),
    (4,6) : ((0,0),),
    (4,7) : ((0,0),),
    (4,8) : ((0,0),),
    (4,9) : ((0,0),),
    (4,10): None,
    (4,11) : ((0,0),),

    
    (5,5) : ((0,0),(1,1)),
    (5,6) : ((0,0),),
    (5,7) : ((0,1),),
    (5,8) : ((0,0),),
    (5,9) : ((0,0),),
    (5,10): ((0,0),),
    (5,11): None,
    
    (6,6) : ((0,0),(1,1)),
    (6,7) : ((0,0),),
    (6,8) : ((0,0),),
    (6,9) : ((0,0),),
    (6,10): ((0,0),),
    (6,11): ((0,0),),
    
    (7,7) : ((0,0),(1,1)),
    (7,8) : ((0,0),),
    (7,9) : ((0,0),(1,1)),
    (7,10): ((1,0),),
    (7,11): ((0,0),),
    
    (8,8) : ((0,0),(1,1)),
    (8,9) : ((0,1),),
    (8,10): ((0,0),),
    (8,11): ((0,0),),
    
    (9,9) : ((0,0),(1,1)),
    (9,10): ((1,0),),
    (9,11): ((0,0),),
    
    (10,10) : ((0,0),(1,1)),
    (10,11) : ((0,0),),
    
    (11,11) : ((0,0),(1,1))
}

big_M_self_edges = {
    0 : (1,0),
    1 : (0,1),
    2 : (0,1),
    3 : (1,0),
    4 : (0,1),
    5 : (1,0),
    6 : (0,1),
    7 : (0,1),
    8 : None,
    9 : (0,1),
    10: (1,0),
    11: (0,1)
}

e = edges[(3,5)]
if e is None:
    print('No connections')
else:
    for (i,j) in e:
        print(i, "connected to", j)

0 connected to 0


In [293]:
df_test = df[df['pitch_class'] == 1]
df['pitch_class'].iloc[3]

11

In [287]:
with open ('./test.txt','w') as f:
    print('NODE_CLASSES := 0 1 2 -2 3 4 5 6 7 -7 8 9 -9 10 11;', file=f)
    for k in range(12):
        print('nodes[{}] :='.format(k),file=f)
        df_next = df[df['pitch_class'] == k]
        for i, j in zip(df_next.id, df_test.pitch_num):
            print(tuple([i,j,0]),file=f)
        print(';',file=f)
        if (k in [2,7,9]):
            print('nodes[{}] :='.format(-k),file=f)
            for i, j in zip(df_next.id, df_test.pitch_num):
                print(tuple([i,j,1]),file=f)
            print(';',file=f)
    print('EDGES :=')
    for idx in range(len(df)):
        print('(',tuple([df['id'].iloc[idx],0,0]),', *)')
        for offset in [-2,-1,1,2]:
            idx_pc = df['pitch_class'].iloc[idx]
            shift = idx + offset
            if ((shift >= 0) & (shift < len(df))):
                if (df['part'].iloc[shift] == df['part'].iloc[idx]):            
                    shift_pc = df['pitch_class'].iloc[shift]
                    

            

EDGES :=
( (4594432376, 0, 0) , *)
( (4594430080, 0, 0) , *)
( (4591103280, 0, 0) , *)
( (4591102944, 0, 0) , *)
( (4591103336, 0, 0) , *)
( (4591103056, 0, 0) , *)
( (4591103784, 0, 0) , *)
( (4591103560, 0, 0) , *)
( (4591101656, 0, 0) , *)
( (4591102216, 0, 0) , *)
( (4591100480, 0, 0) , *)
( (4591100704, 0, 0) , *)
( (4591100872, 0, 0) , *)
( (4591101152, 0, 0) , *)
( (4591100368, 0, 0) , *)
( (4590305744, 0, 0) , *)
( (4590307480, 0, 0) , *)
( (4590305968, 0, 0) , *)
( (4590264896, 0, 0) , *)
( (4590266296, 0, 0) , *)
( (4590268256, 0, 0) , *)
( (4590224888, 0, 0) , *)
( (4590226288, 0, 0) , *)
( (4590178600, 0, 0) , *)
( (4590180280, 0, 0) , *)
( (4590181232, 0, 0) , *)
( (4590141392, 0, 0) , *)
( (4590139376, 0, 0) , *)
( (4590140832, 0, 0) , *)
( (4590140720, 0, 0) , *)
( (4597394120, 0, 0) , *)
( (4597393616, 0, 0) , *)
( (4597394680, 0, 0) , *)
( (4597393504, 0, 0) , *)
( (4597393000, 0, 0) , *)
( (4597393280, 0, 0) , *)
( (4597395128, 0, 0) , *)
( (4595874616, 0, 0) , *)
( (

In [288]:
with open('./test.txt') as f:
    for a in f:
        print(a)

NODE_CLASSES := 0 1 2 -2 3 4 5 6 7 -7 8 9 -9 10 11;

nodes[0] :=

;

nodes[1] :=

(4594432376, 0, 0)

(4591103336, 0, 0)

(4591103784, 0, 0)

(4591102216, 0, 0)

(4590224888, 0, 0)

(4590226288, 0, 0)

(4590181232, 0, 0)

(4595760096, 0, 0)

(4595760600, 0, 0)

(4595760768, 0, 0)

(4596558592, 0, 0)

(4596559488, 0, 0)

(4596559208, 0, 0)

(4596557472, 0, 0)

(4596556296, 0, 0)

(4597269672, 0, 0)

(4597272472, 0, 0)

(4597270680, 0, 0)

(4595617576, 0, 0)

(4595614440, 0, 0)

(4595614664, 0, 0)

(4595614104, 0, 0)

(4512372160, 0, 0)

(4512371432, 0, 0)

(4512371544, 0, 0)

(4512374232, 0, 0)

(4596549056, 0, 0)

(4601599312, 0, 0)

(4594228864, 0, 0)

(4594225784, 0, 0)

(4595718352, 0, 0)

(4595719976, 0, 0)

(4595716728, 0, 0)

;

nodes[2] :=

(4596558312, 0, 0)

(4596558032, 0, 0)

(4596555960, 0, 0)

(4597270120, 0, 0)

(4595615224, 0, 0)

(4595617464, 0, 0)

(4512372608, 0, 0)

(4512373784, 0, 0)

(4512371712, 0, 0)

(4589965664, 0, 0)

(4595718688, 0, 0)

(4595716952, 0, 0)

;


In [289]:
df_naturals = df[(df['pitch class'] == 2) | (df['pitch class'] == 7) | (df['pitch class'] == 9)]
df_naturals.tail()

KeyError: 'pitch class'

In [290]:
with open('./spell.run'):
    df_naturals df[df['pitch'] ]

SyntaxError: invalid syntax (<ipython-input-290-00d214b1ea97>, line 2)